In [11]:
import pandas as pd
ingr= pd.read_csv('tabellen/data_ingr.csv')
recipes=pd.read_csv('tabellen/Recepten_kiezen_g.csv')
recipes

,Unnamed: 0,title,persons,time,calories,stars,url,image,Unnamed: 8
0,Kika:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 wel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Courgette carbonara,4.0,20.0,575.0,0.0,https://www.ah.nl/allerhande/recept/R-R1188954...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
3,0,Mexicaanse taco's met vis en zoete aardappel,4.0,20.0,655.0,0.0,https://www.ah.nl/allerhande/recept/R-R1188828...,https://static.ah.nl/static/recepten/img_09292...,NaN
4,0,Gegrilde-paprikasalade met feta en munt,4.0,15.0,205.0,0.0,https://www.ah.nl/allerhande/recept/R-R1188715...,https://static.ah.nl/static/recepten/img_09094...,NaN
...,...,...,...,...,...,...,...,...,...
131,0,Rodekoolsteak met blauwaderkaas en truffel,4.0,15.0,550.0,0.0,https://www.ah.nl/allerhande/recept/R-R1189726...,https://static.ah.nl/static/recepten/img_10034...,NaN
132,0,Vegan wraps met tofu en rodekoolsalade,4.0,20.0,670.0,0.0,https://www.ah.nl/allerhande/recept/R-R1195491...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
133,0,Ovenschotel met hachee rodekool en puree,4.0,30.0,630.0,0.0,https://www.ah.nl/allerhande/recept/R-R1192021...,https://static.ah.nl/static/recepten/img_12333...,NaN
134,0,Aardappel en zuurkool,4.0,15.0,405.0,0.0,https://www.ah.nl/allerhande/recept/R-R758540/...,https://static.ah.nl/static/recepten/img_00767...,NaN


In [117]:
for i in range(0, len(recipes)):
    ingr[(ingr['recipe']==recipes['title'].iloc[i])]

,Unnamed: 0,recipe,ingredient,quantity,unit
20402,29667,Courgette carbonara,riso pasta,200.0,g
20403,29668,Courgette carbonara,courgette,4.0,NaN
20404,29669,Courgette carbonara,biologische spekreepjes,200.0,g
20405,29670,Courgette carbonara,ongezouten roomboter,30.0,g
20406,29671,Courgette carbonara,tarwebloem,30.0,g
20407,29672,Courgette carbonara,halfvolle melk,250.0,ml
20408,29673,Courgette carbonara,geraspte belegen 30+ kaas,150.0,g


In [118]:
ingr

,Unnamed: 0,recipe,ingredient,quantity,unit
0,0,Kruidnoten met choco-discodip,melkchocolade,100.0,g
1,1,Kruidnoten met choco-discodip,kruidnoten,100.0,g
2,2,Kruidnoten met choco-discodip,discodip,2.0,el
3,3,Kruidnoten in marsepein,blanke marsepein,150.0,g
4,4,Kruidnoten in marsepein,ongezouten roomboter,15.0,g
...,...,...,...,...,...
49814,71804,Biefstuk met rodewijnsaus en ham,olijfolie,4.0,el
49815,71805,Biefstuk met rodewijnsaus en ham,biefstukken,4.0,NaN
49816,71806,Biefstuk met rodewijnsaus en ham,boter,25.0,g
49817,71807,Biefstuk met rodewijnsaus en ham,serranoham,4.0,plakken


In [ ]:
onze = ingr[ingr['recipe'].isin(recipes['title'])]